## Labwork 2

## Parameter estimation in signal processing

## Contents

1. Estimation under additive normal noise
2. Exponential case
3. Estimator comparison

We consider the relaxation signal of a physical process :

$$
\begin{equation*}
s_{i}=s_{0} \exp ^{-\alpha_{0} i}, i \in[0, N-1] \tag{2.1}
\end{equation*}
$$

The objective here is to estimate $\alpha_{0}$ from a noisy signal. We consider successively two types of noises: normal (Gaussian additive) and exponential (multiplicative) noises.

## 1 Estimation under additive normal noise

We first assume that the experiment is perturbed by an additive white normal noise.

- P1 Give the expression of the log-likelihood function $\ell(\alpha)$, then find the expression of the maximum likelihood estimator of $\alpha_{0}$. Can we give an analytical form?

To compute the log-likelihhod, we need to find an expression of the probability law of the noisy signal. Here in the gaussian additive noise, the resulting signal is simply a gaussian signal centered on the true signal and with the variance of the noise.

$$
\begin{align*}
P(x_i) &= \frac{1}{\sqrt(2\pi)\sigma} \times exp\left(-\frac{(s_i-s_0 \times exp(-\alpha_0 i))^2}{2\sigma^2}\right) \\
l_\alpha &= -\frac{N}{2}\times log(2\pi\sigma^2)-\frac{1}{2\sigma^2} \sum_{i=0}^{N-1}\left[s_i-s_0 \times exp(\alpha_0 i)\right]
\end{align*}
$$

The MLE is then obtained for $ \partial l_\alpha / \partial \alpha = 0 $ and should atteins the Cramer Rao Lower Bound (CRLB) defined by $ \partial^2 l_\alpha / \partial^2 \alpha = 0 $ :

$$
\begin{equation*}
\frac{\partial l_\alpha}{\partial \alpha} = \frac{s_0}{\sigma^2}\sum_{i=0}^{N-1}\left[i\times exp(-\alpha_0 i)\times\left(s_0\times exp(-\alpha_0 i)-s_i\right)\right]
\end{equation*}
$$

- P2 Give the expression of the Cramer-Rao Lower Bound (CRLB) for the estimation of $\alpha_{0}$. What does the ratio $s_{0} / \sigma$ represent physically? 

$$
\begin{equation*}
\frac{\partial^2 l_\alpha}{\partial^2 \alpha} = \frac{s_0}{\sigma^2}\sum_{i=0}^{N-1}\left[i^2\times exp(-\alpha_0 i)\times\left(s_i-2\times s_0\times exp(-\alpha_0 i)\right)\right]
\end{equation*}
$$

In some cases (which ones?) the expression of the CRLB can be simplified using:

$$
\sum_{n=0}^{+\infty} n^{2} q^{n}=\frac{q(1+q)}{(1-q)^{3}}, q<1
$$

$\rightsquigarrow$ Design a matlab function which gives a realization (length $N=100$ ) of the noisy signal. The arguments will be $s_{0}, \alpha_{0}$ and $\sigma$, the standard deviation of noise. You can choose the parameters as $\alpha_{0}=0.1, s_{0} / \sigma=10$.

$\rightsquigarrow$ Design a function which calculate the log-likelihood function $\ell(\alpha)$ using the previous realization. $\alpha$ should be an input parameter of the function.

$\rightsquigarrow \operatorname{Plot} \ell(\alpha)$ for $\alpha \in\left[\begin{array}{ll}0 & 1\end{array}\right]$.

$\rightsquigarrow$ Plot again this curve using other realizations.


In [19]:
import numpy as np
from math import pi, log
import plotly.graph_objects as go

def signal(amplitude, X):
    """
    returns [ num_realizations*[time] ]
    """

    # Générer le signal exponentiel
    clean_signal = amplitude * np.exp(-X)

    return clean_signal

def noisySignal(amplitude, X, noise_std):
    """
    returns [ num_realizations*[time] ]
    """

    # Générer le signal exponentiel
    clean_signal = amplitude * np.exp(-X)
    
    # Ajouter du bruit gaussien
    noise = np.random.normal(0, noise_std, size=time.shape) 

    res = clean_signal + noise

    return res

def nNoisySignal(amplitude, decay_rate, noise_std, time, num_realizations):
    """
    returns [ num_realizations*[time] ]
    """

    L = []

    for i in range(num_realizations):
        res = noisySignal(amplitude, decay_rate, noise_std, time)
        L.append(res)

    return L

def loglikelihood(realization, model):
    """
    Returns scalar.
    """

    sigma = np.std(realization)
    residuals = realization - model # [ num_realizations*[time] ] - [time]
    N = len(realization)

    LLH = -N/2 * log(2 * pi * sigma**2) - 1 /2 /sigma**2 * np.sum(residuals**2, axis=1)

    return LLH

if __name__ == "__main__":

    # Paramètres

    # Tirer 100 valeurs entières aléatoires
    time = np.linspace(0, 99, 100)
    decay_rate = np.linspace(0, 1, 100)                # Taux de décroissance

    X = np.outer(time, decay_rate)

    amplitude = 1.0                  # Amplitude du signal exponentiel
    r = 10 # s_0 / sigma
    noise_std = amplitude / r                 # Écart-type du bruit

    num_realizations = 100
    clean = signal(amplitude, X)[:,10]
    noisy = noisySignal(amplitude, X, noise_std)[:,10]

    noisySignals = []
    signals = []
    LLHs = []

    for i in range(10):
        noisy_signal = noisySignal(amplitude, X, noise_std)
        clean_signal = signal(amplitude, X)

        noisySignals.append(noisy_signal)
        signals.append(clean_signal)
        LLHs.append(loglikelihood(noisy_signal, clean_signal))

    # Create the plot
    fig = go.Figure()

    # Add error bars
    fig.add_trace(go.Scatter(
        x=time,
        y=noisy,
        mode='markers+lines',
        name='Noisy signal',
        #error_y=dict(type='data', array=std_devs, visible=True),
        marker=dict(size=10)
    ))

        # Add error bars
    fig.add_trace(go.Scatter(
        x=time,
        y=clean,
        mode='markers+lines',
        name='Clean signal',
        #error_y=dict(type='data', array=std_devs, visible=True),
        marker=dict(size=10)
    ))

    # Customize layout
    fig.update_layout(
        title='Noisy and model signal, alpha = 0.1',
        xaxis_title='time',
        yaxis_title='Signal',
        #yaxis=dict(title='LLHs', range=[0, max(LLHs) + 1]),
        #xaxis=dict(title='Decay Rate (alpha)', tickvals=decay_rate),
        showlegend=True
    )

    # Show the plot
    fig.show()


    # Create the plot
    fig2 = go.Figure()

    # Add error bars
    for i in range(10):
        fig2.add_trace(go.Scatter(
            x=decay_rate,
            y=LLHs[i],
            mode='markers+lines',
            name=f'LLhs number {i}',
            #error_y=dict(type='data', array=std_devs, visible=True),
            marker=dict(size=10)
        ))

    # Customize layout
    fig2.update_layout(
        title='LLHs for Different Realizations',
        xaxis_title='Decay Rate (alpha)',
        yaxis_title='LLHs',
        #yaxis=dict(title='LLHs', range=[0, max(LLHs) + 1]),
        #xaxis=dict(title='Decay Rate (alpha)', tickvals=decay_rate),
        showlegend=True
    )

    # Show the plot
    fig2.show()




Q1 What are your comments?
$\rightsquigarrow$ Design a function which finds the maximum likelihood estimator from a realization. You can use the matlab functions fzero or fminsearch.
$\rightsquigarrow \quad$ Estimate the statistical mean and variance of the estimator with a Monte Carlo simulation.

Q2 Is the estimator biased? Does it reach the CRLB?

## Useful commands/tips:

- fzero: tries to find a zero of a function. The syntax to use is: fzero( $@(x)$ fonc (x, a_1, a_2), x0) to find the zero of the function fonc ( $\alpha$, a_1 , a_2) according to $\alpha$. The a_1, a_2 parameter stay constant during optimization. XO is the initial value.
- fminsearch: attempts to find a local minimizer of a function. The syntax to use is: fminsearch(@(x) fonc(x, a_1, a_2), X0);


## 2 Exponential case

We now consider that the signal (2.1) is pertubed by exponential noise. Remember that for such process (with mean $I$ ) the probability density is :

$$
P(x)=\frac{1}{I} \exp \left[-\frac{x}{I}\right]
$$

P3 Express the maximum likelihood estimator of $\alpha_{0}$. Is it the same as the Gaussian case?

P4 Express the CRLB for $\alpha_{0}$ estimation. In order to obtain an analytical form, we can use the result:

$$
\sum_{n=0}^{N} n^{2}=\frac{N(N+1)(2 N+1)}{6}
$$

What are the parameters the CRLB depends on?
$\rightsquigarrow$ Design a Matlab function which gives a realization of the stochastic process.

Q3 On which parameters does it depend?
$\rightsquigarrow$ Design a function which finds the maximum likelihood estimator from an experiment. We can use the Matlab functions fzero or fminsearch.
$\rightsquigarrow$ Estimate statistical mean and variance of the estimator with a Monte Carlo simulation.

Q4 Is the estimator biased? Does it reach the CRLB?

## Useful commands/tips:

$\mathrm{R}=\mathrm{randexp}(1, \mathrm{n}, \mathrm{lambda})$ returns pseudo-random values drawn from an exponential distribution with means stored in the vector $(1 \times n)$ lambda.
10 LABWORK 2: Parameter estimation in signal processing (Sept., 23th)

## 3 Estimator comparison

$\rightsquigarrow \quad$ Apply the maximum likelihood estimator adapted to the normal noise to signal pertubed by exponential noise. Plot the estimate for several realizations. Estimate the variance.

Q5 What are your comments?